# 🧪 Симуляция данных магнитометра

Этот ноутбук моделирует данные эталонного и неоткалиброванного магнитометра для тестирования алгоритмов калибровки.

In [22]:
import numpy as np
import pandas as pd
from scipy.spatial.transform import Rotation as R
import matplotlib.pyplot as plt
%matplotlib inline

### Смещение (bias), матрица искажения (A), чувствительность и шумы

In [23]:
bias = np.array([100.0, -250.0, 600.0])  # нТл

A = np.array([
    [1.05, 0.02, 0.01],
    [0.01, 0.98, -0.03],
    [-0.01, 0.02, 1.02]
])

sensitivity = np.array([0.95, 1.03, 1.00]) 
noise_std = np.array([0.5, 0.5, 0.5])
ref_noise_std = 0.2
detection_threshold = 1.0
rotate_axes = True

### Направление и величина геомагнитного поля

In [24]:
B_magnitude = 49500  # нТл
theta_deg = 70.5
phi_deg = 11.98

theta = np.radians(theta_deg)
phi = np.radians(phi_deg)

B_real = np.array([
    B_magnitude * np.cos(theta) * np.cos(phi),
    B_magnitude * np.cos(theta) * np.sin(phi),
    B_magnitude * np.sin(theta)
])

print("B_real =", B_real)

B_real = [16163.56093597  3429.77431316 46660.75380906]


### Углы поворота стола

In [25]:
angles_deg = np.arange(0, 360, 10)
angles_rad = np.radians(angles_deg)


### Симуляция поворота на поворотном столе

In [26]:
M_ref = np.array([
    [
        B_real[0] * np.cos(a) - B_real[1] * np.sin(a),
        B_real[0] * np.sin(a) + B_real[1] * np.cos(a),
        B_real[2]
    ]
    for a in angles_rad
])

### Дополнительный поворот осей

In [27]:
if rotate_axes:
    rot_matrix = R.from_euler('xyz', [2, -1, 3], degrees=True).as_matrix()
    M_ref = M_ref @ rot_matrix.T

### Добавление шума и порог чувствительности

In [28]:
ref_noise = np.random.normal(0, ref_noise_std, M_ref.shape)
M_ref_noisy = M_ref + ref_noise
M_ref_final = np.where(np.abs(M_ref_noisy) < detection_threshold, 0, M_ref_noisy)

### Симуляция неоткалиброванного магнитометра

In [29]:
bias_temp = np.tile(bias, (len(angles_deg), 1))
sensitivity_temp = np.tile(sensitivity, (len(angles_deg), 1))

M_raw = M_ref_final * sensitivity_temp
M_raw = (M_raw @ A.T) + bias_temp

noise = np.random.normal(0, noise_std, M_ref.shape)
M_raw += noise

### Сохранение в CSV

In [30]:
df = pd.DataFrame({
    "angle_deg": angles_deg,
    "raw_x": M_raw[:, 0],
    "raw_y": M_raw[:, 1],
    "raw_z": M_raw[:, 2],
    "ref_x": M_ref_final[:, 0],
    "ref_y": M_ref_final[:, 1],
    "ref_z": M_ref_final[:, 2],
})

df.to_csv("calibration_data.csv", index=False)
print("✅ Данные сохранены в calibration_data.csv")

✅ Данные сохранены в calibration_data.csv


In [31]:
df.head()

,angle_deg,raw_x,raw_y,raw_z,ref_x,ref_y,ref_z
0,0,15815.533142,1108.431404,48476.466263,15229.907990,2599.976939,47026.978944
1,10,14889.660762,3826.155047,48624.516700,14244.411872,5304.950649,47108.128407
2,20,13503.881128,6327.896249,48757.056860,12803.660651,7798.166646,47175.006453
3,30,11703.742544,8535.600790,48871.352166,10952.349532,10003.962270,47225.042984
4,40,9540.372841,10380.778676,48963.709018,8745.777023,11854.656974,47257.306754
